In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

# 1. Scraping the Data from the Wikipedia Page

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html5lib')

### **1.1 - Cleaning and wrangling the data**

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

### **1.2 - Checking for not assigned data**

In [4]:
print(f"There are {df.loc[df['Borough'] == 'Not assigned'].shape[0]} Not assigned Boroughs")

There are 0 Not assigned Boroughs


In [5]:
print(f"There are {df.loc[df['Neighborhood'] == 'Not assigned'].shape[0]} Not assigned Neighborhoods")

There are 0 Not assigned Neighborhoods


In [6]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


### **1.3 - Testing if the example matches the data**

In [7]:
example_postalcode = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

df.loc[df['PostalCode'].isin(example_postalcode)]

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M1B,Scarborough,"Malvern, Rouge"
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
23,M4G,East York,Leaside
24,M5G,Downtown Toronto,Central Bay Street
27,M2H,North York,Hillcrest Village
32,M1J,Scarborough,Scarborough Village
50,M9L,North York,Humber Summit
54,M4M,East Toronto,Studio District
71,M1R,Scarborough,"Wexford, Maryvale"


### **1.4 - Printing the shape of the data**

In [8]:
df.shape

(103, 3)

# 2. Getting the geographical coordinates from the data

You can get the data either from geocoder or the dataframe.

In [9]:
# import geocoder

# for index, content in df.iterrows():
#     coordinates = None
#     while(coordinates is None):
#         coordinates = geocoder.arcgis(f'{content[0]}, Toronto, Ontario').latlng
#     latitude = coordinates[0]
#     longitude = coordinates[1]
#     df.loc[index, 'Latitude'] = latitude
#     df.loc[index, 'Longitude'] = longitude

Loading the dataframe with the coordinates

In [10]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the two dataframes

In [11]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df = df.join(coordinates.set_index('PostalCode'), on='PostalCode')

In [12]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### **2.1 - Testing if the example matches the data**

In [13]:
example_postalcode = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

df.loc[df['PostalCode'].isin(example_postalcode)]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
23,M4G,East York,Leaside,43.709060,-79.363452
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
27,M2H,North York,Hillcrest Village,43.803762,-79.363452
32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
50,M9L,North York,Humber Summit,43.756303,-79.565963
54,M4M,East Toronto,Studio District,43.659526,-79.340923
71,M1R,Scarborough,"Wexford, Maryvale",43.750071,-79.295849


# 3. Explore and cluster the neighborhoods in Toronto

### **3.1 - Creating map of all the city with neighborhoods superimposed on top**

Getting the location of toronto with geopy

In [14]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto-explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### **3.2 - Selecting only the boroughs that contain the word Toronto**

In [16]:
df_toronto_borough = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_borough.shape

(39, 5)

### **3.3 - Creating map of toronto borough with neighborhoods superimposed on top**

In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_borough['Latitude'], df_toronto_borough['Longitude'], df_toronto_borough['Borough'], df_toronto_borough['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### **3.4. Use the Foursquare API to explore the neighborhoods**

In [62]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [19]:
df_toronto_borough.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


### 3.5 - Let's get all the venues by borough and neighborhood

In [20]:
def getNearbyVenues(boroughs, neighborhoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for borough, neighborhood, lat, lng in zip(boroughs, neighborhoods, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
borough_neighborhood_venues = getNearbyVenues(boroughs=df_toronto_borough['Borough'],
                                             neighborhoods=df_toronto_borough['Neighborhood'],
                                             latitudes=df_toronto_borough['Latitude'],
                                             longitudes=df_toronto_borough['Longitude'])

In [22]:
borough_neighborhood_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


### 3.6 - Checking the size of the dataframe

In [23]:
borough_neighborhood_venues.shape

(1583, 8)

Let's check how many venues were returned for each borough and neighborhood

In [24]:
borough_neighborhood_venues.groupby(['Borough', 'Neighborhood']).count()

eaches                                                             4   
                       The Danforth West, Riverdale                                           43   
East Toronto Business  Enclave of M4L                                                         17   
East York/East Toronto The Danforth  East                                                      3   
West Toronto           Brockton, Parkdale Village, Exhibition Place                           23   
                       Dufferin, Dovercourt Village                                           15   
                       High Park, The Junction South                                          25   
                       Little Portugal, Trinity                                               43   
                       Parkdale, Roncesvalles                                                 14   
                       Runnymede, Swansea                                                     35   

                                                                           Neighborhood Longitude  \
Borough                Neighborhood                                                                 
Central Toronto        Davisville                                                              34   
                       Davisville North                                                        11   
                       Forest Hill North & West                                                 4   
                       Lawrence Park                                                            3   
                       Moore Park, Summerhill East                                              1   
                       North Toronto West                                                      17   
                       Roselawn                                                                 2   
                       Summerhill West, Rathnelly, South Hill, Forest ...                      14   
                       The Annex, North Midtown, Yorkville                                     19   
Downtown Toronto       Berczy Park                                                             60   
                       CN Tower, King and Spadina, Railway Lands, Harb...                      15   
                       Central Bay Street                                                      62   
                       Christie                                                                16   
                       Church and Wellesley                                                    76   
                       Commerce Court, Victoria Hotel                                         100   
                       First Canadian Place, Underground city                                 100   
                       Garden District, Ryerson                                               100   
                       Harbourfront East, Union Station, Toronto Islands                      100   
                       Kensington Market, Chinatown, Grange Park                               63   
                       Regent Park, Harbourfront                                               46   
                       Richmond, Adelaide, King                                                96   
                       Rosedale                                                                 4   
                       St. James Town                                                          84   
                       St. James Town, Cabbagetown                                             46   
                       Toronto Dominion Centre, Design Exchange                               100   
                       University of Toronto, Harbord                                          32   
Downtown Toronto Stn A Enclave of M5E                                                         100   
East Toronto           India Bazaar, The Beaches West                                          20 

**Let's find out how many unique categories can be curated from all the returned venues**

In [25]:
print(f'There are {borough_neighborhood_venues["Venue Category"].nunique()} uniques categories.')

There are 229 uniques categories.


### 3.7 - Analyze each Neighborhood and Venue

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(borough_neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = borough_neighborhood_venues['Borough'] 
toronto_onehot['Neighborhoods'] = borough_neighborhood_venues['Neighborhood'] 
toronto_onehot['Venue'] = borough_neighborhood_venues['Venue'] 

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Neighborhoods,Venue,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Downtown Toronto,"Regent Park, Harbourfront",Roselle Desserts,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Downtown Toronto,"Regent Park, Harbourfront",Tandem Coffee,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,"Regent Park, Harbourfront",Cooper Koo Family YMCA,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Downtown Toronto,"Regent Park, Harbourfront",Body Blitz Spa East,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Downtown Toronto,"Regent Park, Harbourfront",Impact Kitchen,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [27]:
toronto_onehot.shape

(1583, 232)

#### Next, let's group rows by borough and neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
toronto_grouped = toronto_onehot.groupby(['Borough', 'Neighborhoods']).mean().reset_index()
toronto_grouped

,Borough,Neighborhoods,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Central Toronto,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.029412,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
1,Central Toronto,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
2,Central Toronto,Forest Hill North & West,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
3,Central Toronto,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
4,Central Toronto,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
5,Central Toronto,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.058824,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.058824
6,Central Toronto,Roselawn,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
7,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.071429,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.071429,0.000000,0.000000
8,Central Toronto,"The Annex, North Midtown, Yorkville",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
9,Downtown Toronto,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.016667,0.00000,0.000000,0.000000,0.000000


Let's confirm the new size

In [29]:
toronto_grouped.shape

(39, 231)

### Let's print each neighborhood along with the top 5 most common venues

In [30]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0         Pizza Place  0.09
1        Dessert Shop  0.09
2      Sandwich Place  0.09
3  Italian Restaurant  0.06
4         Coffee Shop  0.06


----Davisville North----
                  venue  freq
0  Gym / Fitness Center  0.09
1        Breakfast Spot  0.09
2                   Gym  0.09
3                 Hotel  0.09
4     Food & Drink Shop  0.09


----Forest Hill North & West----
              venue  freq
0              Park  0.25
1             Trail  0.25
2     Jewelry Store  0.25
3  Sushi Restaurant  0.25
4    Adult Boutique  0.00


----Lawrence Park----
            venue  freq
0            Park  0.33
1        Bus Line  0.33
2     Swim School  0.33
3  Adult Boutique  0.00
4          Museum  0.00


----Moore Park, Summerhill East----
                venue  freq
0          Restaurant   1.0
1      Adult Boutique   0.0
2         Music Venue   0.0
3         Men's Store   0.0
4  Mexican Restaurant   0.0


----North Toronto West----
            

**Let's put that into a pandas dataframe**

First, let's write a function to sort the venues in descending order.

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough', 'Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Coffee Shop,Gym,Café,Sushi Restaurant,Pharmacy,Diner
1,Central Toronto,Davisville North,Gym / Fitness Center,Breakfast Spot,Gym,Hotel,Food & Drink Shop,Dog Run,Department Store,Park,Playground,Sandwich Place
2,Central Toronto,Forest Hill North & West,Park,Trail,Jewelry Store,Sushi Restaurant,Adult Boutique,Museum,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
3,Central Toronto,Lawrence Park,Park,Bus Line,Swim School,Adult Boutique,Museum,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant
4,Central Toronto,"Moore Park, Summerhill East",Restaurant,Adult Boutique,Music Venue,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark


In [33]:
neighborhoods_venues_sorted.shape

(39, 12)

### 3.8. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [54]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(['Borough', 'Neighborhoods'], 1)

# run k-means clustering
kmeans = KMeans(init='k-means++', n_clusters=kclusters, random_state=1, max_iter=1000).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 4, 2, 0, 3, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [55]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

toronto_merged = df_toronto_borough.copy()

# merge toronto_grouped with df_toronto_borough to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(['Borough'], axis=1).set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Café,Pub,Park,Bakery,Breakfast Spot,Theater,Event Space,Brewery,Chocolate Shop
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Café,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Ramen Restaurant,Electronics Store
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cosmetics Shop,Gastropub,Cocktail Bar,Lingerie Store,Park,Department Store,Restaurant,Creperie
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Health Food Store,Trail,Pub,Monument / Landmark,Museum,Moving Target,Movie Theater,Moroccan Restaurant,Adult Boutique
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Beer Bar,Cheese Shop,Seafood Restaurant,Pharmacy,Farmers Market,Nightclub


**Finally, let's visualize the resulting clusters**

In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.9 - Examining the Clusters

#### Cluster - 1

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Café,Pub,Park,Bakery,Breakfast Spot,Theater,Event Space,Brewery,Chocolate Shop
1,Downtown Toronto,0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Café,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Ramen Restaurant,Electronics Store
2,Downtown Toronto,0,Coffee Shop,Café,Cosmetics Shop,Gastropub,Cocktail Bar,Lingerie Store,Park,Department Store,Restaurant,Creperie
3,East Toronto,0,Neighborhood,Health Food Store,Trail,Pub,Monument / Landmark,Museum,Moving Target,Movie Theater,Moroccan Restaurant,Adult Boutique
4,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Beer Bar,Cheese Shop,Seafood Restaurant,Pharmacy,Farmers Market,Nightclub
5,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Burger Joint,Salad Place,Bubble Tea Shop,Spa,Sushi Restaurant
6,Downtown Toronto,0,Grocery Store,Café,Park,Candy Store,Nightclub,Baby Store,Italian Restaurant,Athletics & Sports,Coffee Shop,Restaurant
7,Downtown Toronto,0,Coffee Shop,Café,Thai Restaurant,Restaurant,Gym,Clothing Store,Deli / Bodega,Pizza Place,Sushi Restaurant,Lounge
8,West Toronto,0,Pharmacy,Bakery,Music Venue,Middle Eastern Restaurant,Liquor Store,Café,Furniture / Home Store,Supermarket,Bar,Bank
10,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Pizza Place,Restaurant,Brewery,Scenic Lookout,Sporting Goods Shop


#### Cluster - 2

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,East York/East Toronto,1,Intersection,Convenience Store,Park,Music Venue,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant
33,Downtown Toronto,1,Park,Playground,Trail,Adult Boutique,Music Venue,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant


#### Cluster - 3

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,2,Restaurant,Adult Boutique,Music Venue,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark


#### Cluster - 4

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,3,Garden,Pool,Adult Boutique,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark


#### Cluster - 5

In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,4,Park,Bus Line,Swim School,Adult Boutique,Museum,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant
